# ForceSMIP Minimal Example: Ridge Regression

The goal of this notebook is to demonstrate the use of ridge regression to estimate the forced and unforced components of global mean surface temperature variability and change. This is the place where you introductory information about your project, and also the place to work on your group project. 

Outline:

* Notes on setup with conda
* Import of key packages
* Surveying the ForceSMIP provided data
* Looping over each available climate model to:
* Finding and looping over all available ensemble members
* Loading data for each member
* Calculating the annual average departure (from 1900 - 1949)
* Storing full fields for a subset of the ensemble (to be used for our training/prediction)
* Calculating and storing the global mean time series for all members
* Calculate the forced response for each model by taking the ensemble mean of all global mean time series

* Train a regression model based on a random grid cell for reference and comparison
* Create predictor and predictand matrices
* Train Ridge Regression Model
* See how well model works on ensemble members not used in training dataset

* Run prediction with ridge regression for the Tier 1 test dataset

Reference: The notebook implements the main method from this paper, which was presented in the lecture: Sippel et al., (2020), *Nature Climate Change*, Climate change now detectable from any single day of weather at global scale https://www.nature.com/articles/s41558-019-0666-7. 

#### Recommended: Choose your (existing) kernel

Before you can start, navigate to Kernel-> Change Kernel, and choose the Kernel "Teaching - R_ForceSMIP3". Alternatively, you can create your own conda environement (follow instructions below), but this is not necessary.

#### Alternative: Create your own conda environment for your project

Before you can start, we need to create an environment, which runs your R version and all packages you need. To create the conda environment, please follow the following commands:

1. Move to File -> Launcher, and open a Terminal.

2. Load the conda environment manager, by typing: `module load conda`

3. Create a new environment with a name (e.g., 'R_ForceSMIP') with the respective dependencies installed: 
`conda create --name R_ForceSMIP3 r r-base=4.2.3 r-essentials r-ncdf4 r-ncdf4.helpers r-raster r-irkernel r-matrixStats r-rnaturalearth r-rnaturalearthdata r-doParallel r-foreach --y` (this will run a couple of minutes)

4. When the command has run, restart you Jupyter Lab Session, and select Kernel->Change Kernel, and choose your new Kernel `R [conda env:.conda-R_ForceSMIP]`

Now, you're ready to go!

If you need to install further packages later, you can easily do so in the Terminal session, e.g.: 
`source activate R_ForceSMIP`, followed by, for example: `conda install r-ncdf4` (which would install the ncdf4 R package)
To learn more about conda, you can use the Cheatsheeet here: https://docs.conda.io/projects/conda/en/4.6.0/_downloads/52a95608c49671267e40c689e0bc00ca/conda-cheatsheet.pdf

In [2]:
# for internal use:
# cp /home/sippels/IAC_lectures/2023_ForceSMIP/Ridge_Regression/ridge_regression.ipynb /net/iacftp/ftp/pub_read/sippels/ForceSMIP/notebook_RR/ridge_regression.ipynb
# Path to publicly readable FRP directory: ftp://iacftp.ethz.ch/pub_read/sippels/ForceSMIP/notebook_RR/ridge_regression.ipynb

In [1]:
########################################################################
## 0.a. required packages
########################################################################

library(ncdf4)
library(ncdf4.helpers)
library(data.table)
library(raster)
library(rnaturalearth)
library(rnaturalearthdata)
library(matrixStats)
library(ggplot2)
library(glmnet)
library(magrittr)

library(doParallel)
library(foreach)

R.version

Loading required package: sp

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Support for Spatial objects (`sp`) will be deprecated in {rnaturalearth} and will be removed in a future release of the package. Please use `sf` objects with {rnaturalearth}. For example: `ne_download(returnclass = 'sf')`


Attaching package: ‘rnaturalearthdata’


The following object is masked from ‘package:rnaturalearth’:

    countries110


Loading required package: Matrix

Loaded glmnet 4.1-8


Attaching package: ‘magrittr’


The following object is masked from ‘package:raster’:

    extr

               _                           
platform       x86_64-conda-linux-gnu      
arch           x86_64                      
os             linux-gnu                   
system         x86_64, linux-gnu           
status                                     
major          4                           
minor          2.3                         
year           2023                        
month          03                          
day            15                          
svn rev        83980                       
language       R                           
version.string R version 4.2.3 (2023-03-15)
nickname       Shortstop Beagle            

In [ ]:
########################################################################
## 0.b. required functions
########################################################################

# Gets the raw data of specified variable out of the netcdf file
extract_data <- function(in_file, var = NULL){
  ncfile <- nc_open(in_file)
  ncdata <- ncvar_get(nc = ncfile, varid = var)
  nc_close(ncfile)
  return(ncdata)
}

# Extracts annual data from the monthly netCDF files 
# (monthly resolution not needed for the ridge forced resp. estimate)
extract_data_annual <- function(in_file, temp.file = c("/home/sippels/test.nc"), var = NULL){
  print(paste("processed: ", in_file))
  system(paste("cdo -O -yearmean ", in_file," ", temp.file, sep=""))
  ncfile <- nc_open(temp.file)
  ncdata <- ncvar_get(nc = ncfile, varid = var)
  nc_close(ncfile)
  file.remove(temp.file)
  return(ncdata)
  #system(paste("rm ", temp.file, sep=""))
}


# Helper function for the areaw_avg function, produces weights for every grid cell
get_area_weights <- function(res = 2.5){
  raster.template = raster(res = res, xmn = 0, xmx=360, ymn = -90, ymx=90)
  y <- dim(raster.template)[1]
  x <- dim(raster.template)[2]
  areaw=c(matrix(values(raster::area(raster.template)), x,y)[,y:1]) / sum(c(matrix(values(raster::area(raster.template)), x,y)[,y:1]))
  return(areaw)
}

# Calculates an area weighted average based on a specific mask. 
# usage here is not going to require masking (i think) so the mask will usually be the entire globe
areaw_avg <- function(X, mask, apply_mask = T){
  area_weghts <- get_area_weights()
  area_weghts_masked <- area_weghts[mask] * 10368/length(mask)
  if(apply_mask){
    avg <-  c(X[,mask] %*% area_weghts_masked)
  }else{
    avg <-  c(X %*% area_weghts_masked)
  }
  return(avg)
}

# Helper function to get lat and lon as coordinates of each grid cell's center (requires next function convert.to.Pacific)
get_lonlatDT <- function(res = 2.5, SREX = F){
  require(raster)
  r_template = raster(res = res, xmn = -180, xmx=180, ymn = -90, ymx=90)
  r_template <- convert.to.pacific(r_template)
  lonlatDT <- data.table(
    lat = c(matrix(coordinates(r_template)[,2],144, 72)[,72:1]),
    lon = coordinates(r_template)[,1])
  lonlatDT[lon > 180, lon := lon -360]
  lonlatDT[, lonlat := paste(lon,lat)]
  
  if(SREX == T){
    load("/net/xenon/climphys_backedup/maegli/Misc/SREX/IPCC-WGI-reference-regions-v4_R.rda")
    
    for(i in 1:length( names(IPCC_WGI_reference_regions_v4@polygons))){
      region <- names(IPCC_WGI_reference_regions_v4@polygons)[i]
      region_ix <- which(names(IPCC_WGI_reference_regions_v4@polygons) == region)
      
      x <- IPCC_WGI_reference_regions_v4@polygons[[region_ix]]@Polygons[[1]]@coords[,1]
      y <- IPCC_WGI_reference_regions_v4@polygons[[region_ix]]@Polygons[[1]]@coords[,2]
      
      SREX_mask <- which(point.in.polygon(lonlatDT$lon, lonlatDT$lat, x, y) == 1)
      
      lonlatDT[SREX_mask, SREX := region]
    } 
  }
  
  
  return(lonlatDT)
}


# Helper function to make everything Pacific-centered (raster package left bound is at 0 Meridian..)
convert.to.pacific <- function(raster.in) {
  rl <- crop(raster.in, extent(c(xmin(raster.in)+180,xmax(raster.in),ymin(raster.in), ymax(raster.in))))
  
  # set extent from rr to rleft:
  rr <- (crop(raster.in, extent(c(xmin(raster.in),xmax(raster.in)-180,ymin(raster.in), ymax(raster.in)))))
  extent(rr) <- extent(c(xmin(rr)+360,xmax(rr)+360,ymin(raster.in), ymax(raster.in)))
  
  r.out = merge(rr,rl)
  names(r.out) = names(raster.in)
  
  return(r.out)
}


# subtract reference period based on monthly data frame:
center.XAX_mon <- function(XAX, ref.period.years = 1950:2000) {
  
  XAX_norm = XAX
  XAX_norm$X[,] = NA
  
  mod.un = unique(as.character(XAX$M$mod))
  
  for (mon in 1:12) {
    for (m in 1:length(mod.un)) {
      print(paste(mod.un[m], "- Month:", mon))
      mod.ix = which(XAX$M$mod == mod.un[m] & XAX$M$mon == mon)
      ref.mod.ix = which(XAX$M$mod == mod.un[m] & XAX$M$year %in% ref.period.years & XAX$M$mon == mon)
      
      clim_mean = apply(X = XAX$X[ref.mod.ix,], MARGIN = 2, FUN = mean)
      XAX_norm$X[mod.ix,] = t(t(XAX$X[mod.ix,]) - clim_mean)
    }
  }
  return(XAX_norm)
}

# subtract reference period based on annual data:
center.XAX_ann <- function(XAX, ref.period.years = 1950:2000) {
  
  XAX_norm = XAX
  XAX_norm$X[,] = NA
  
  mod.un = unique(as.character(XAX$M$mod))
  
  for (m in 1:length(mod.un)) {
    #print(paste(mod.un[m], "- Month:", mon))
    mod.ix = which(XAX$M$mod == mod.un[m])
    ref.mod.ix = which(XAX$M$mod == mod.un[m] & XAX$M$year %in% ref.period.years)
    
    clim_mean = apply(X = XAX$X[ref.mod.ix,], MARGIN = 2, FUN = mean)
    XAX_norm$X[mod.ix,] = t(t(XAX$X[mod.ix,]) - clim_mean)
  }
  return(XAX_norm)
}





In [ ]:
########################################################################
## 01. Training Data Read-In  
########################################################################

input_dir <- "/net/krypton/climdyn_nobackup/FTP/ForceSMIP/Training/Amon/tas/"
#read files stored in input directory
input_files <- list.files(input_dir, recursive = T)

# !!!IMPORTANT!!! Define the temporary file in a directory where you have writing permissions. 
# I.e., replace 'sippels' with *your* User name: 
temp.file = "/home/sippels/temp.nc"  

#creating a data frame containg the meta data stored in the file strings 
# Later we add also the timestamps of the netcdf files in input_files
split_strings <- strsplit(input_files, "_")
meta_data <- data.frame(matrix(unlist(split_strings), ncol = max(lengths(split_strings)), byrow = TRUE))
colnames(meta_data) <-  c("var", "res", "mod", "scen1", "scen2", "rest")

# Get full directory name to all input files
full_input_files <- sapply(input_files, function(x) paste0(input_dir, x))

# Create a list ForceSMIP_train, which will contain the training data X, the metadata M, and the corresponding response Y. 
# Each one of X, M and Y will correspond to one list entry. ForceSMIP thus is a list with three entries, X, M and Y.  
ForceSMIP_train <- list()

# Creating M.. 
meta_list <- list()
for(i in 1:nrow(meta_data)){
  #print(i)
  timestamps <- substr(nc.get.time.series(nc_open(full_input_files[i])),1, 10)
  meta_list[[i]] <- data.frame(matrix(rep(as.character(meta_data[i,]), length(timestamps)), ncol = ncol(meta_data), byrow = TRUE))
  colnames(meta_list[[i]]) <- c("var", "res", "mod", "scen1", "scen2", "rest")
  #meta_list[[i]][, rest := unlist(rest)]
  meta_list[[i]]$ens.mem = substr(meta_list[[i]]$rest, 1, 15)
  meta_list[[i]]$time = timestamps
  meta_list[[i]]$year = as.integer(substr(meta_list[[i]]$time, 1,4))
  #meta_list[[i]]$mon =  as.integer(substr(meta_list[[i]]$time, 6,7))
  #meta_list[[i]]$day =  as.integer(substr(meta_list[[i]]$time, 9,10))
  # for annual data:
  meta_list[[i]] = meta_list[[i]][seq(6, 1716, 12),]
  meta_list[[i]]$res = "ann"
}

ForceSMIP_train$M<- data.frame(do.call("rbind", meta_list))

In [ ]:
# Explore the meta data:
str(ForceSMIP_train$M)


In [ ]:
# Creating X, meaning processing+reading the actual *annual* data:
# Be PATIENT, this step may take up to 5 minutes!!

raw_data <- list()
for (i in 1:length(full_input_files)) {
  print(i)
  raw_data[[i]] = extract_data_annual(in_file = full_input_files[i], temp.file = temp.file, var = "tas")
}

ForceSMIP_train$X <- do.call("rbind", lapply(raw_data, function(x) t(apply(x,3,as.vector))))

# Subtract reference period, so that each grid cell is centered around its own 1950-2000 mean:
# (This is necessary to ensure the there are no model-specific mean-state biases in the training data, 
# i.e., a region that is typically too warm or too cold)
ForceSMIP_train <- center.XAX_ann(XAX = ForceSMIP_train, ref.period.years = 1950:2000)

# Now add also the response Y to the list ForceSMIP_train
ForceSMIP_train$Y <- list()

#caclucalte forced response (ensemble mean global mean surface temperature)
ForceSMIP_train$Y$GMST <- areaw_avg(ForceSMIP_train$X, 1:dim(ForceSMIP_train$X)[2])
ForceSMIP_train$Y$FR <- data.table(GMST = ForceSMIP_train$Y$GMST,
                                   year = ForceSMIP_train$M$year,
                                   mod = ForceSMIP_train$M$mod)[
                                     , FR := mean(GMST), by = list(mod, year)][,FR]
#clean up
rm(raw_data)
gc()

In [ ]:

########################################################################
## 01. Predicting the FR by regressing a random grid cell onto the FR
########################################################################

# Split the data into a training and a testing data set. We use one random ensemble member for testing, 
# and all remaining ens. members for training. We train on all models simulatenously 
train_ix <- which(ForceSMIP_train$M$ens.mem != "r1011.001i1p1f1")
test_ix <- which(ForceSMIP_train$M$ens.mem == "r1011.001i1p1f1")

#randomaly selects a grid cell from which to predict the forced response
set.seed(1234)
loc <- sample(1:dim(ForceSMIP_train$X)[2], 1)

y <- ForceSMIP_train$Y$FR[train_ix]
x <- ForceSMIP_train$X[train_ix,loc]

fit <- lm(y~ x)

#test the model with the left out ensemble meber and generate scatterplot
test_data <- data.frame(x = ForceSMIP_train$X[test_ix,loc])
y_pred <- predict(fit, test_data)

data.table(pred = y_pred,
           true = ForceSMIP_train$Y$FR[test_ix],
           year = ForceSMIP_train$M$year[test_ix]) %>% 
  ggplot()+
  geom_point(aes(pred, true, color = year))+
  scale_color_viridis_c()


# very poor performance...



In [ ]:
########################################################################
## 02a. Ridge regression (takes some time to run)
########################################################################

registerDoParallel(cores=5)

# define 20 members per model for training:
mod.un = unique(ForceSMIP_train$M$mod)

train.ens.mem = c(sapply(X = mod.un, FUN = function(mod) { unique(ForceSMIP_train$M$ens.mem[which(ForceSMIP_train$M$mod == mod)])[1:20] }))
train.mod = c(rep("CanESM5", 20), rep("CESM2", 20), rep("MIROC-ES2L", 20), rep("MIROC6", 20), rep("MPI-ESM1-2-LR", 20))
train_ix <- which(ForceSMIP_train$M$ens.mem %in% train.ens.mem & ForceSMIP_train$M$mod %in% train.mod)


test.ens.mem = c(sapply(X = mod.un, FUN = function(mod) { unique(ForceSMIP_train$M$ens.mem[which(ForceSMIP_train$M$mod == mod)])[21:25] }))
test.mod = c(rep("CanESM5", 5), rep("CESM2", 5), rep("MIROC-ES2L", 5), rep("MIROC6", 5), rep("MPI-ESM1-2-LR", 5))
test_ix <- which(ForceSMIP_train$M$ens.mem %in% test.ens.mem & ForceSMIP_train$M$mod %in% test.mod)

# We now defines the cross validation folds, used for the lambda estimation
# Each member is it's own cross validation fold, leading to a high number of folds
# This makes the fitting procedure a bit slow 
crossclass <- as.numeric(as.factor(ForceSMIP_train$M$mod[train_ix]))

#fit the ridge model
ridge_fit <- cv.glmnet(x = ForceSMIP_train$X[train_ix, ],
                       y = ForceSMIP_train$Y$FR[train_ix],
                       foldid = crossclass,
                       alpha = 0, parallel = T)

### Plot the ridge regression predictions against underlying data

Explore the data and predictions.

In [ ]:
########################################################################
## 02b. Explore the results of the ridge regression fit
########################################################################


#generate scatter plot of predicted vs target
pred_table <- data.table(pred = c(predict(ridge_fit, ForceSMIP_train$X[test_ix, ], s = "lambda.1se")),
                         true = ForceSMIP_train$Y$FR[test_ix],
                         year = ForceSMIP_train$M$year[test_ix])

#scatter plot
ggplot(pred_table)+
  geom_point(aes(pred, true))

#time series
pred_table %>% melt(id.vars = "year") %>% 
  ggplot()+
  geom_line(aes(year, value, color = variable))

plot(pred_table$year, pred_table$true, col = "black")
points(pred_table$year, pred_table$pred, col = "red")


#borders for map plot
world <- rnaturalearth::ne_countries(scale = "medium", returnclass = "sf")

sds_tas <- c(matrixStats::colSds(ForceSMIP_train$X))

world <- rnaturalearth::ne_countries(scale = "medium", returnclass = "sf")

lonlatDT <- get_lonlatDT(res = 2.5, SREX = F)
lonlatDT[, betas := ridge_fit$glmnet.fit$beta[,ridge_fit$index[2]]]
lonlatDT[, sds := sds_tas]
lonlatDT[, scaled_betas := betas*sds_tas]

#map plot
ggplot(lonlatDT) +
  geom_tile(aes(lon, lat, fill = scaled_betas))+
  geom_sf(data = world, fill=alpha("lightgrey", 0), color="grey50") +
  ylab("") + xlab("") +
  scale_fill_gradient2(high = "#c90202", mid = "white", low ="#0237c9", name= "Scaled Coefficients") +
  theme(panel.grid.major = element_blank()) 

### Example application of trained ridge regression model onto Tier 1 Evaluation to estimate forced response

In [ ]:
########################################################################
## 05a. Read-In of Out-of-sample(!!) Tier1 testing data
########################################################################

input_dir <- "/net/krypton/climdyn_nobackup/FTP/ForceSMIP/Evaluation-Tier1/Amon/tas/"
#read files stored in input directory
input_files <- list.files(input_dir, recursive = T)

# !!!IMPORTANT!!! Define the temporary file in a directory where you have writing permissions. 
# I.e., replace 'sippels' with *your* User name: 
temp.file = "/home/sippels/temp.nc"

#creating a data frame containg the meta data stored in the file strings as well as the timestamps of the netcdf files in input_files
split_strings <- strsplit(input_files, "_")
meta_data <- data.frame(matrix(unlist(split_strings), ncol = max(lengths(split_strings)), byrow = TRUE))
colnames(meta_data) <-  c("var", "res", "mod")

full_input_files <- sapply(input_files, function(x) paste0(input_dir, x))
ForceSMIP_test <- list()
meta_list <- list()
for(i in 1:nrow(meta_data)){
  print(i)
  timestamps <- substr(nc.get.time.series(nc_open(full_input_files[i])),1, 10)
  meta_list[[i]] <- data.frame(matrix(rep(as.character(meta_data[i,]), length(timestamps)), ncol = ncol(meta_data), byrow = TRUE))
  colnames(meta_list[[i]]) <- c("var", "res", "mod")
  #meta_list[[i]][, rest := unlist(rest)]
  meta_list[[i]]$time = timestamps
  meta_list[[i]]$year = as.integer(substr(meta_list[[i]]$time, 1,4))
  #meta_list[[i]]$mon =  as.integer(substr(meta_list[[i]]$time, 6,7))
  #meta_list[[i]]$day =  as.integer(substr(meta_list[[i]]$time, 9,10))
  # for annual data:
  meta_list[[i]] = meta_list[[i]][seq(6, 73*12, 12),]
  meta_list[[i]]$res = "ann"
}

ForceSMIP_test$M<- data.frame(do.call("rbind", meta_list))

# ForceSMIP_train$X[1,] == ForceSMIP_train$X[144,]
# ForceSMIP_train$M[1,] ; ForceSMIP_train$M[144,]
# raw_data[[1]] == raw_data[[2]]

#reading the actual data:
raw_data <- list()
for (i in 1:length(full_input_files)) {
  print(i)
  raw_data[[i]] = extract_data_annual(in_file = full_input_files[i], temp.file = temp.file, var = "tas")
}

ForceSMIP_test$X <- do.call("rbind", lapply(raw_data, function(x) t(apply(x,3,as.vector))))
#subtract reference period:
ForceSMIP_test <- center.XAX_ann(XAX = ForceSMIP_test, ref.period.years = 1950:2000)

ForceSMIP_test$Y <- list()
#caclucalte forced response (ensemble mean global mean surface temperature)
ForceSMIP_test$Y$GMST <- areaw_avg(ForceSMIP_test$X, 1:dim(ForceSMIP_test$X)[2])

#clean up
rm(raw_data)
gc()

In [ ]:

########################################################################
## 05b. Get prediction for out-of-sample testing data:
########################################################################

pred_table <- data.table(pred = c(predict(ridge_fit, ForceSMIP_test$X, s = "lambda.1se")),
                         var = ForceSMIP_test$M$var,
                         res = ForceSMIP_test$M$res,
                         mod = ForceSMIP_test$M$mod,
                         year = ForceSMIP_test$M$year)


### Some Discussion Questions (from Steven Po-Chedley's PLS notebook):

* Why do you think the coefficient maps look the way they do?
* Note that we trained and validated on the same five global climate models. How well do you think this would this work on out-of-sample models? On observations?
* Observations have missing data. How would we apply this model-trained Ridge Regression model on observations?
* The hyperparameter is chosen by cross-validation. What would happen if we chose a smaller or larger value of the hyperparameter $\lambda$?
* The ForceSMIP protocol asks for spatially resolved predictions of the forced response (e.g., [time, lat, lon]). How could you extend this notebook to go from global mean predictions to spatially resolved predictions?
Do you think this approach would work well for other variables of interest?

## Now it's your turn!

In [ ]:
# start coding here ...